In [25]:
%%bash
cat > main.py << 'EOF'
import argparse
import random
import time
import os
import csv
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import wandb

# ─── CharVocab & Dataset ─────────────────────────────────────────────────────
class CharVocab:
    def __init__(self, filepaths: List[str]):
        self.rom_char2idx: Dict[str,int] = {}
        self.dev_char2idx: Dict[str,int] = {}
        self.rom_idx2char: Dict[int,str] = {}
        self.dev_idx2char: Dict[int,str] = {}
        self._build_vocab(filepaths)

    def _build_vocab(self, filepaths: List[str]):
        rom_chars = set()
        dev_chars = set()
        for fp in filepaths:
            with open(fp, "r", encoding="utf-8") as f:
                reader = csv.reader(f, delimiter="\t")
                for row in reader:
                    if len(row) < 2:
                        continue
                    devanagari = row[0].strip()
                    roman      = row[1].strip()
                    rom_chars.update(list(roman))
                    dev_chars.update(list(devanagari))

        PAD, SOS, EOS = "<pad>", "<sos>", "<eos>"

        all_rom = [PAD, SOS, EOS] + sorted(rom_chars)
        for i, ch in enumerate(all_rom):
            self.rom_char2idx[ch] = i
            self.rom_idx2char[i] = ch

        all_dev = [PAD, SOS, EOS] + sorted(dev_chars)
        for i, ch in enumerate(all_dev):
            self.dev_char2idx[ch] = i
            self.dev_idx2char[i] = ch

        self.rom_pad_idx = self.rom_char2idx[PAD]
        self.rom_sos_idx = self.rom_char2idx[SOS]
        self.rom_eos_idx = self.rom_char2idx[EOS]

        self.dev_pad_idx = self.dev_char2idx[PAD]
        self.dev_sos_idx = self.dev_char2idx[SOS]
        self.dev_eos_idx = self.dev_char2idx[EOS]

    @property
    def rom_vocab_size(self) -> int:
        return len(self.rom_char2idx)

    @property
    def dev_vocab_size(self) -> int:
        return len(self.dev_char2idx)

    def roman_to_indices(self, s: str) -> List[int]:
        return [self.rom_sos_idx] + [self.rom_char2idx[ch] for ch in s] + [self.rom_eos_idx]

    def dev_to_indices(self, s: str) -> List[int]:
        return [self.dev_sos_idx] + [self.dev_char2idx[ch] for ch in s] + [self.dev_eos_idx]

    def indices_to_dev(self, idxs: List[int]) -> str:
        chars = []
        for i in idxs:
            if i in (self.dev_sos_idx, self.dev_eos_idx, self.dev_pad_idx):
                continue
            chars.append(self.dev_idx2char[i])
        return "".join(chars)


def read_tsv(path: str) -> List[Tuple[str, str]]:
    """
    Expects each line of the TSV to be:
      Devanagari_word    Roman_word    <something_to_ignore>
    We only need (Roman, Devanagari) for training.
    """
    pairs = []
    with open(path, "r", encoding="utf-8") as f:
        reader = csv.reader(f, delimiter="\t")
        for row in reader:
            # If there are fewer than 2 columns, skip
            if len(row) < 2:
                continue

            # Unpack: Dev is first column, Roman is second, ignore anything else
            devana = row[0].strip()
            roman  = row[1].strip()

            if not roman or not devana:
                continue
            # Append (roman, devana)—this matches our CharVocab convention
            pairs.append((roman, devana))
    return pairs


class TransliterationDataset(Dataset):
    def __init__(self, filepath, vocab):
        super().__init__()
        self.pairs = read_tsv(filepath)
        self.vocab = vocab

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        roman, devanagari = self.pairs[idx]
        roman_idxs = self.vocab.roman_to_indices(roman)
        dev_idxs = self.vocab.dev_to_indices(devanagari)
        return torch.tensor(roman_idxs, dtype=torch.long), torch.tensor(dev_idxs, dtype=torch.long)

    @staticmethod
    def collate_fn(batch):
        roman_seqs, dev_seqs = zip(*batch)
        max_rom_len = max(len(x) for x in roman_seqs)
        max_dev_len = max(len(x) for x in dev_seqs)
        rom_padded = []
        dev_padded = []
        for r, d in zip(roman_seqs, dev_seqs):
            pad_r = torch.cat([r, r.new_full((max_rom_len - len(r),), r.new_tensor(0))])
            pad_d = torch.cat([d, d.new_full((max_dev_len - len(d),), d.new_tensor(0))])
            rom_padded.append(pad_r)
            dev_padded.append(pad_d)
        return torch.stack(rom_padded), torch.stack(dev_padded)


# ─── Encoder & Decoder ────────────────────────────────────────────────────────
class Encoder(nn.Module):
    def __init__(self, input_vocab_size, embed_dim, hidden_size, num_layers, cell_type, dropout):
        super().__init__()
        self.embed = nn.Embedding(input_vocab_size, embed_dim, padding_idx=0)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type.upper()
        if self.cell_type == "RNN":
            self.rnn = nn.RNN(
                embed_dim, hidden_size, num_layers=num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0.0,
            )
        elif self.cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_dim, hidden_size, num_layers=num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0.0,
            )
        elif self.cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_dim, hidden_size, num_layers=num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0.0,
            )
        else:
            raise ValueError(f"Unsupported cell type: {cell_type}")

    def forward(self, x):
        emb = self.embed(x)
        if self.cell_type == "LSTM":
            outputs, (h_n, c_n) = self.rnn(emb)
            return outputs, (h_n, c_n)
        else:
            outputs, h_n = self.rnn(emb)
            return outputs, h_n


class Decoder(nn.Module):
    def __init__(self, output_vocab_size, embed_dim, hidden_size, num_layers, cell_type, dropout):
        super().__init__()
        self.embed = nn.Embedding(output_vocab_size, embed_dim, padding_idx=0)
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type.upper()
        if self.cell_type == "RNN":
            self.rnn = nn.RNN(
                embed_dim, hidden_size, num_layers=num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0.0
            )
        elif self.cell_type == "GRU":
            self.rnn = nn.GRU(
                embed_dim, hidden_size, num_layers=num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0.0
            )
        elif self.cell_type == "LSTM":
            self.rnn = nn.LSTM(
                embed_dim, hidden_size, num_layers=num_layers,
                batch_first=True, dropout=dropout if num_layers > 1 else 0.0
            )
        else:
            raise ValueError(f"Unsupported cell type: {cell_type}")

        self.out = nn.Linear(hidden_size, output_vocab_size)

    def forward(self, tgt_seq, hidden, cell=None, teacher_forcing_ratio=0.0):
        B, T = tgt_seq.size()
        outputs = torch.zeros(B, T, self.out.out_features, device=tgt_seq.device)
        input_step = tgt_seq[:, 0]
        if self.cell_type == "LSTM":
            h = hidden       # h_n: (num_layers, B, hidden_size)
            c = cell         # c_n: (num_layers, B, hidden_size)
        else:
            h = hidden
            c = None

        for t in range(1, T):
            emb_t = self.embed(input_step).unsqueeze(1)
            if self.cell_type == "LSTM":
                out_step, (h, c) = self.rnn(emb_t, (h, c))
            else:
                out_step, h = self.rnn(emb_t, h)
                c = None

            logits = self.out(out_step.squeeze(1))
            outputs[:, t, :] = logits

            teacher_force = (torch.rand(1).item() < teacher_forcing_ratio)
            top1 = logits.argmax(dim=1)
            next_input = tgt_seq[:, t] if teacher_force else top1
            input_step = next_input.view(-1)

        return outputs

    def beam_search_decode(self, encoder_hidden, encoder_cell, max_len, dev_sos_idx, dev_eos_idx, beam_size):
        hidden, cell = encoder_hidden, encoder_cell
        live = [([dev_sos_idx], hidden, cell, 0.0)]
        completed = []

        for _ in range(max_len):
            new_hyps = []
            for (seq, h, c, score) in live:
                last_token = seq[-1]
                if last_token == dev_eos_idx:
                    completed.append((seq, h, c, score))
                    continue

                inp = torch.tensor([last_token], dtype=torch.long, device=h.device).unsqueeze(0)
                emb_t = self.embed(inp)

                if self.cell_type == "LSTM":
                    out_t, (h2, c2) = self.rnn(emb_t, (h, c))
                else:
                    out_t, h2 = self.rnn(emb_t, h)
                    c2 = None

                logits = self.out(out_t.squeeze(1))
                log_probs = F.log_softmax(logits, dim=1)

                topk_logprobs, topk_indices = torch.topk(log_probs, k=beam_size, dim=1)
                topk_logprobs = topk_logprobs.squeeze(0).tolist()
                topk_indices = topk_indices.squeeze(0).tolist()
                for lp, idx in zip(topk_logprobs, topk_indices):
                    new_hyps.append((seq + [idx], h2, c2, score + lp))

            # Keep only top‐k
            new_hyps = sorted(new_hyps, key=lambda x: x[3], reverse=True)[:beam_size]
            live = new_hyps
            if all(hyp[0][-1] == dev_eos_idx for hyp in live):
                completed.extend(live)
                break

        if not completed:
            completed = live
        best = max(completed, key=lambda x: x[3])
        return best[0]


class Seq2Seq(nn.Module):
    def __init__(self, encoder: Encoder, decoder: Decoder, device: torch.device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src: torch.Tensor, tgt: torch.Tensor, teacher_forcing_ratio: float = 0.5) -> torch.Tensor:
        """
        src: (B, T_src)
        tgt: (B, T_tgt)   # including <sos> … <eos>
        Returns:
          logits: (B, T_tgt, V_out)
        """
        B, T_src = src.size()
        outputs = torch.zeros(B, tgt.size(1), self.decoder.out.out_features, device=self.device)

        # 1) Run the encoder
        if self.encoder.cell_type == "LSTM":
            enc_outputs, (h_n, c_n) = self.encoder(src)
        else:
            enc_outputs, h_n = self.encoder(src)
            c_n = None

        # 2) Transform encoder's hidden‐state to match decoder layers
        enc_layers = self.encoder.num_layers
        dec_layers = self.decoder.num_layers
        hidden_size = self.encoder.hidden_size

        if self.encoder.cell_type == "LSTM":
            if enc_layers >= dec_layers:
                h_dec = h_n[-dec_layers:]
                c_dec = c_n[-dec_layers:]
            else:
                num_missing = dec_layers - enc_layers
                zeros_h = torch.zeros(num_missing, B, hidden_size, device=self.device)
                zeros_c = torch.zeros(num_missing, B, hidden_size, device=self.device)
                h_dec = torch.cat([zeros_h, h_n], dim=0)
                c_dec = torch.cat([zeros_c, c_n], dim=0)
            dec_hidden = h_dec
            dec_cell = c_dec
        else:
            if enc_layers >= dec_layers:
                h_dec = h_n[-dec_layers:]
            else:
                num_missing = dec_layers - enc_layers
                zeros_h = torch.zeros(num_missing, B, hidden_size, device=self.device)
                h_dec = torch.cat([zeros_h, h_n], dim=0)
            dec_hidden = h_dec
            dec_cell = None

        # 3) Run decoder (with teacher forcing)
        logits = self.decoder(
            tgt_seq=tgt,
            hidden=dec_hidden,
            cell=dec_cell,
            teacher_forcing_ratio=teacher_forcing_ratio,
        )
        return logits

    @torch.no_grad()
    def predict(self, src: torch.Tensor, max_len: int, dev_sos_idx: int, dev_eos_idx: int, beam_size: int = 1) -> List[List[int]]:
        """
        Greedy (beam_size=1) or beam search decoding for batch size=1. Returns a list of decoded indices.
        """
        self.eval()
        if self.encoder.cell_type == "LSTM":
            _, (h_n, c_n) = self.encoder(src)
        else:
            _, h_n = self.encoder(src)
            c_n = None

        enc_layers = self.encoder.num_layers
        dec_layers = self.decoder.num_layers
        hidden_size = self.encoder.hidden_size
        B = 1

        if self.encoder.cell_type == "LSTM":
            if enc_layers >= dec_layers:
                h_dec = h_n[-dec_layers:]
                c_dec = c_n[-dec_layers:]
            else:
                num_missing = dec_layers - enc_layers
                zeros_h = torch.zeros(num_missing, B, hidden_size, device=self.device)
                zeros_c = torch.zeros(num_missing, B, hidden_size, device=self.device)
                h_dec = torch.cat([zeros_h, h_n], dim=0)
                c_dec = torch.cat([zeros_c, c_n], dim=0)
            hidden_state = h_dec
            cell_state = c_dec
        else:
            if enc_layers >= dec_layers:
                h_dec = h_n[-dec_layers:]
            else:
                num_missing = dec_layers - enc_layers
                zeros_h = torch.zeros(num_missing, B, hidden_size, device=self.device)
                h_dec = torch.cat([zeros_h, h_n], dim=0)
            hidden_state = h_dec
            cell_state = None

        if beam_size == 1:
            seq = [dev_sos_idx]
            hidden_, cell_ = hidden_state, cell_state
            for _ in range(max_len):
                last_token = torch.tensor([seq[-1]], dtype=torch.long, device=self.device).unsqueeze(0)
                emb = self.decoder.embed(last_token)
                if self.decoder.cell_type == "LSTM":
                    out, (h_next, c_next) = self.decoder.rnn(emb, (hidden_, cell_))
                    hidden_, cell_ = h_next, c_next
                else:
                    out, h_next = self.decoder.rnn(emb, hidden_)
                    hidden_, cell_ = h_next, None

                logits = self.decoder.out(out.squeeze(1))
                next_token = logits.argmax(dim=1).item()
                seq.append(next_token)
                if next_token == dev_eos_idx:
                    break
            return [seq]
        else:
            best_seq = self.decoder.beam_search_decode(
                encoder_hidden=hidden_state,
                encoder_cell=cell_state,
                max_len=max_len,
                dev_sos_idx=dev_sos_idx,
                dev_eos_idx=dev_eos_idx,
                beam_size=beam_size,
            )
            return [best_seq]


# ─── train / evaluate ───────────────────────────────────────────────────────
def char_accuracy(
    logits: torch.Tensor,
    target: torch.Tensor,
    pad_idx: int,
    sos_idx: int,
    eos_idx: int
) -> Tuple[int, int]:
    """
    Compute (num_correct_chars, num_valid_chars) ignoring <pad>, <sos>, and <eos>.
    logits: (B, T, V)    – raw decoder logits
    target: (B, T)       – ground‐truth indices (including <sos>, <eos>, <pad>)
    pad_idx, sos_idx, eos_idx: indices for the special tokens

    Returns:
       (correct_count, valid_count)
    """
    with torch.no_grad():
        B, T, V = logits.size()
        pred = logits.argmax(dim=2)   # (B, T)

        # Mask out positions where target is pad, sos, or eos
        ignore_mask = (
            (target != pad_idx) &
            (target != sos_idx) &
            (target != eos_idx)
        )  # (B, T)

        correct = ((pred == target) & ignore_mask).sum().item()
        valid   = ignore_mask.sum().item()
        return correct, valid


def train_one_epoch(
    model: Seq2Seq,
    iterator: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.CrossEntropyLoss,
    pad_idx: int,
    sos_idx: int,
    eos_idx: int,
    device: torch.device,
    teacher_forcing_ratio: float,
) -> Tuple[float, float, float]:
    """
    Runs one epoch of training, returning:
      (train_loss, train_char_acc, train_word_acc)

    – train_char_acc is computed as (total_correct_chars / total_valid_chars)
    – train_word_acc is (num_exact_word_matches / total_words)
    """
    model.train()
    epoch_loss = 0.0

    # For micro‐average char‐accuracy:
    total_correct_chars = 0
    total_valid_chars   = 0

    # For word‐accuracy (we’ll count how many sequences are exactly correct)
    total_exact_words = 0
    total_words       = 0

    for src, tgt in iterator:
        src = src.to(device, non_blocking=True)  # (B, T_src)
        tgt = tgt.to(device, non_blocking=True)  # (B, T_tgt)
        B, T_tgt = tgt.size()

        optimizer.zero_grad()
        # Forward pass
        output_logits = model(src, tgt, teacher_forcing_ratio=teacher_forcing_ratio)
        # output_logits: (B, T_tgt, V_out)

        # 1) Compute token‐level (character) loss
        V = output_logits.size(-1)
        loss = criterion(output_logits.view(-1, V), tgt.view(-1))

        # 2) Compute char‐level correct/valid counts (ignore special tokens)
        correct, valid = char_accuracy(
            output_logits, tgt,
            pad_idx, sos_idx, eos_idx
        )
        total_correct_chars += correct
        total_valid_chars   += valid

        # 3) Compute word‐level accuracy for this batch
        #    We already have output_logits. Let’s get preds:
        with torch.no_grad():
            pred_inds = output_logits.argmax(dim=2)  # (B, T_tgt)

            # For each example, ignore index 0 (<sos>). Then check that
            #   for every position t where tgt[b,t] != pad_idx,
            #   pred_inds[b,t] == tgt[b,t].
            pred_trim = pred_inds[:, 1:]
            tgt_trim  = tgt[:,      1:]
            mask_trim = (tgt_trim != pad_idx)  # which positions to verify

            # A word is correct if for all non‐pad positions, pred == tgt
            match_trim    = (pred_trim == tgt_trim) | (~mask_trim)
            exact_matches = match_trim.all(dim=1)  # (B,)
            total_exact_words += exact_matches.sum().item()
            total_words       += B

        # 4) Backward + step
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()

        epoch_loss += loss.item()

    # Final epoch‐level metrics:
    train_loss = epoch_loss / len(iterator)
    train_char_acc = total_correct_chars / max(total_valid_chars, 1)
    train_word_acc = total_exact_words   / max(total_words, 1)

    return train_loss, train_char_acc, train_word_acc


def evaluate(
    model: Seq2Seq,
    iterator: DataLoader,
    criterion: nn.CrossEntropyLoss,
    pad_idx: int,
    sos_idx: int,
    eos_idx: int,
    device: torch.device,
    beam_size: int,
    max_dev_len: int,
) -> Tuple[float, float, float]:
    """
    One validation pass (no teacher forcing). Returns:
      (val_loss, val_char_acc, val_word_acc)

    – val_char_acc is (total_correct_chars / total_valid_chars)
    – val_word_acc is (num_exact_word_matches / total_words)
    """
    model.eval()
    epoch_loss = 0.0

    total_correct_chars = 0
    total_valid_chars   = 0
    total_exact_words   = 0
    total_words         = 0

    with torch.no_grad():
        for src, tgt in iterator:
            src = src.to(device, non_blocking=True)
            tgt = tgt.to(device, non_blocking=True)   # (B, T_tgt)
            B, T_tgt = tgt.size()

            # 1) Forward pass (teacher_forcing_ratio=0.0)
            logits = model(src, tgt, teacher_forcing_ratio=0.0)  # (B, T_tgt, V_out)
            V = logits.size(-1)
            loss = criterion(logits.view(-1, V), tgt.view(-1))

            # 2) Character‐level correct/valid counts (ignore special tokens)
            correct, valid = char_accuracy(
                logits, tgt,
                pad_idx, sos_idx, eos_idx
            )
            total_correct_chars += correct
            total_valid_chars   += valid

            # 3) Word‐level accuracy (greedy decode)
            pred_inds = logits.argmax(dim=2)  # (B, T_tgt)
            pred_trim = pred_inds[:, 1:]
            tgt_trim  = tgt[:,      1:]
            mask_trim = (tgt_trim != pad_idx)

            match_trim    = (pred_trim == tgt_trim) | (~mask_trim)
            exact_matches = match_trim.all(dim=1)  # (B,)
            total_exact_words += exact_matches.sum().item()
            total_words       += B

            epoch_loss += loss.item()

    val_loss = epoch_loss / len(iterator)
    val_char_acc = total_correct_chars / max(total_valid_chars, 1)
    val_word_acc = total_exact_words / max(total_words, 1)

    return val_loss, val_char_acc, val_word_acc


# ─── main() with parse_known_args ─────────────────────────────────────────────
def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--data_dir", type=str, default="/content")
    parser.add_argument("--train_file", type=str, default="bn.translit.sampled.train.tsv")
    parser.add_argument("--dev_file", type=str, default="bn.translit.sampled.dev.tsv")
    parser.add_argument("--test_file", type=str, default="bn.translit.sampled.test.tsv")
    parser.add_argument("--emb_size", type=int, default=64)
    parser.add_argument("--hidden_size", type=int, default=64)
    parser.add_argument("--enc_layers", type=int, default=1)
    parser.add_argument("--dec_layers", type=int, default=1)
    parser.add_argument("--cell_type", type=str, default="RNN", choices=["RNN", "GRU", "LSTM"])
    parser.add_argument("--dropout", type=float, default=0.2)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--batch_size", type=int, default=128)
    parser.add_argument("--epochs", type=int, default=10)
    parser.add_argument("--tf_ratio", type=float, default=0.5)
    parser.add_argument("--beam_size", type=int, default=1)
    parser.add_argument("--seed", type=int, default=42)
    parser.add_argument("--max_dev_len", type=int, default=32)
    parser.add_argument("--project_name", type=str, default="Vanilla_RNN")
    parser.add_argument("--run_name", type=str, default=None)

    # ── use parse_known_args to ignore Colab’s extra "-f …json"
    args, _ = parser.parse_known_args()

    random.seed(args.seed)
    torch.manual_seed(args.seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_path = os.path.join(args.data_dir, args.train_file)
    dev_path   = os.path.join(args.data_dir, args.dev_file)
    test_path  = os.path.join(args.data_dir, args.test_file)
    vocab = CharVocab([train_path, dev_path, test_path])

    # Extract special‐token indices for convenience
    pad_idx = vocab.dev_pad_idx
    sos_idx = vocab.dev_sos_idx
    eos_idx = vocab.dev_eos_idx

    train_ds = TransliterationDataset(train_path, vocab)
    dev_ds   = TransliterationDataset(dev_path, vocab)
    test_ds  = TransliterationDataset(test_path, vocab)

    train_loader = DataLoader(
        train_ds,
        batch_size=args.batch_size,
        shuffle=True,
        collate_fn=TransliterationDataset.collate_fn,
        num_workers=2,
        pin_memory=True,
    )
    dev_loader = DataLoader(
        dev_ds,
        batch_size=args.batch_size,
        shuffle=False,
        collate_fn=TransliterationDataset.collate_fn,
        num_workers=2,
        pin_memory=True,
    )
    test_loader = DataLoader(
        test_ds,
        batch_size=1,
        shuffle=False,
        collate_fn=TransliterationDataset.collate_fn,
        num_workers=2,
        pin_memory=True,
    )

    if args.run_name is None:
        args.run_name = (
            f"{args.cell_type}"
            f"_emb{args.emb_size}"
            f"_hid{args.hidden_size}"
            f"_layers{args.enc_layers}x{args.dec_layers}"
            f"_lr{args.lr}"
            f"_tf{args.tf_ratio}"
        )

    wandb.init(
        project=args.project_name,
        name=args.run_name,
        config={
            "emb_size": args.emb_size,
            "hidden_size": args.hidden_size,
            "enc_layers": args.enc_layers,
            "dec_layers": args.dec_layers,
            "cell_type": args.cell_type,
            "dropout": args.dropout,
            "lr": args.lr,
            "batch_size": args.batch_size,
            "epochs": args.epochs,
            "tf_ratio": args.tf_ratio,
            "beam_size": args.beam_size,
        },
    )
    config = wandb.config

    encoder = Encoder(
        input_vocab_size=vocab.rom_vocab_size,
        embed_dim=config.emb_size,
        hidden_size=config.hidden_size,
        num_layers=config.enc_layers,
        cell_type=config.cell_type,
        dropout=config.dropout,
    )
    decoder = Decoder(
        output_vocab_size=vocab.dev_vocab_size,
        embed_dim=config.emb_size,
        hidden_size=config.hidden_size,
        num_layers=config.dec_layers,
        cell_type=config.cell_type,
        dropout=config.dropout,
    )
    model = Seq2Seq(encoder, decoder, device).to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=vocab.dev_pad_idx)
    optimizer = torch.optim.Adam(model.parameters(), lr=config.lr, weight_decay=1e-5)

    best_val_acc = 0.0
    for epoch in range(1, config.epochs + 1):
        start_time = time.time()

        # 1) Train
        train_loss, train_char_acc, train_word_acc = train_one_epoch(
            model=model,
            iterator=train_loader,
            optimizer=optimizer,
            criterion=criterion,
            pad_idx=pad_idx,
            sos_idx=sos_idx,
            eos_idx=eos_idx,
            device=device,
            teacher_forcing_ratio=config.tf_ratio,
        )

        # 2) Evaluate
        val_loss, val_char_acc, val_word_acc = evaluate(
            model=model,
            iterator=dev_loader,
            criterion=criterion,
            pad_idx=pad_idx,
            sos_idx=sos_idx,
            eos_idx=eos_idx,
            device=device,
            beam_size=config.beam_size,
            max_dev_len=args.max_dev_len,
        )

        elapsed = time.time() - start_time

        # 3) Save best model based on val_char_acc
        if val_char_acc > best_val_acc:
            best_val_acc = val_char_acc
            torch.save(model.state_dict(), "best_model.pt")

        # 4) Log to W&B
        wandb.log(
            {
                "epoch": epoch,
                "train_loss": train_loss,
                "train_char_acc": train_char_acc,
                "train_word_acc": train_word_acc,
                "val_loss": val_loss,
                "val_char_acc": val_char_acc,
                "val_word_acc": val_word_acc,
                "epoch_time_sec": elapsed,
            }
        )

        # 5) Print progress
        print(
            f"Epoch {epoch:02d} | "
            f"Train Loss: {train_loss:.4f}, "
            f"Train Char Acc: {train_char_acc:.4f}, "
            f"Train Word Acc: {train_word_acc:.4f} | "
            f"Val Loss: {val_loss:.4f}, "
            f"Val Char Acc: {val_char_acc:.4f}, "
            f"Val Word Acc: {val_word_acc:.4f} | "
            f"Time: {elapsed:.1f}s"
        )

    wandb.finish()


if __name__ == "__main__":
    main()

EOF

In [26]:
%%bash
cat > sweep.yaml << 'EOF'
program: main.py
method: bayes
project: Vanilla_RNN
entity: mrsagarbiswas-iit-madras
metric:
  name: val_char_acc
  goal: maximize
parameters:
  emb_size:
    values: [128, 256]
  hidden_size:
    values: [128, 256]
  enc_layers:
    values: [3, 5]
  epochs:
    values: [10, 15]
  dec_layers:
    values: [1, 2, 3]
  cell_type:
    values: ["RNN", "GRU", "LSTM"]
  dropout:
    values: [0.2, 0.3]
  lr:
    values: [1e-3, 1e-4]
  batch_size:
    values: [32, 64, 128]
  tf_ratio:
    values: [0.3, 0.5, 0.7]
  beam_size:
    values: [3, 5]
EOF


In [ ]:
%%bash
export WANDB_API_KEY="Your-API-Key"
wandb login
SWEEP_ID=$(wandb sweep sweep.yaml)
echo "Sweep ID = $SWEEP_ID"

Sweep ID = 


wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Currently logged in as: mrsagarbiswas (mrsagarbiswas-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Creating sweep from: sweep.yaml
wandb: Creating sweep with ID: lv5aj1ns
wandb: View sweep at: https://wandb.ai/mrsagarbiswas-iit-madras/Vanilla_RNN/sweeps/lv5aj1ns
wandb: Run sweep agent with: wandb agent mrsagarbiswas-iit-madras/Vanilla_RNN/lv5aj1ns


In [28]:
!wandb agent mrsagarbiswas-iit-madras/Vanilla_RNN/lv5aj1ns --count 5

wandb: WARNING Using legacy-service, which is deprecated. If this is unintentional, you can fix it by ensuring you do not call `wandb.require('legacy-service')` and do not set the WANDB_X_REQUIRE_LEGACY_SERVICE environment variable.
wandb: Starting wandb agent 🕵️
2025-05-19 15:26:53,379 - wandb.wandb_agent - INFO - Running runs: []
2025-05-19 15:26:53,765 - wandb.wandb_agent - INFO - Agent received command: run
2025-05-19 15:26:53,765 - wandb.wandb_agent - INFO - Agent starting run with config:
	batch_size: 32
	beam_size: 5
	cell_type: LSTM
	dec_layers: 2
	dropout: 0.4
	emb_size: 256
	enc_layers: 3
	epochs: 10
	hidden_size: 256
	lr: 0.0001
	tf_ratio: 0.5
2025-05-19 15:26:53,766 - wandb.wandb_agent - INFO - About to run command: /usr/bin/env python main.py --batch_size=32 --beam_size=5 --cell_type=LSTM --dec_layers=2 --dropout=0.4 --emb_size=256 --enc_layers=3 --epochs=10 --hidden_size=256 --lr=0.0001 --tf_ratio=0.5
wandb: WARNING Using legacy-service, which is deprecated. If this is un